In [ ]:
#AIMS_essay_codes
'''
Created by Samuel Edet
Date: 4th May, 2017
Project: Recurrent neural network in forecasting S&P 500.
Title: AIMS master essay codes
'''

import numpy as np
np.random.seed(123)

import time

import matplotlib.pyplot as plt
%matplotlib inline

import pandas as pd
from pandas_datareader import data as web
import math

import theano
import theano.tensor as TT

from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, Activation,GRU, SimpleRNN

from sklearn.preprocessing import StandardScaler
from sklearn.cross_validation import train_test_split
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report

import warnings
warnings.simplefilter('ignore')


#uploading the csv file
features = pd.DataFrame.from_csv('features.csv')

#Log returns of the dataset
features= np.log(features / features.shift(1))
features = features.fillna(np.mean(features.ix[:]))
features = features.astype('float32')

#correlation of datassets
features.corr()

#From the correlation table the selected features are 
#SPYt-1, JNJ, IXIC and the different levels of neurons and epoch.
X = features.ix[:,['SPYt-1','JNJ','^IXIC']].values

#To determine the upward or downward movement the log return of the features
X[1:,:]=X[1:,:]-X[0:-1,:]   #differnce between today's return and yesterday's return for the training data set
X[0,:]=0.000001
X_del_1=X[0:-1,:]  #exludes the last row  

#the target stock is SPY
y = features.ix[:,'SPY'].values

#To determine the upward or downward movement the log return of the target
y[1:]=y[1:]-y[0:-1] 
y[0]=.000001

y = np.where(y >= 0.0, 1, 0)
y_del_1=y[1:]      #exludes the first row

#allocating dataset into training and testing
#80% is allocated to training and 20% to testing
#allocation for X,y
X_train, X_test, y_train, y_test = train_test_split(
         X, y, test_size=0.2, random_state=0)        

#standardising training and testing dataset
sc = StandardScaler()
#standardising  for X
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

# reshape input to be [samples, time steps, features]
X_train_std = np.reshape(X_train_std, (X_train_std.shape[0], X_train_std.shape[1], 1))
X_test_std = np.reshape(X_test_std, (X_test_std.shape[0], X_test_std.shape[1], 1))


#-----------------------------------------------
#THE NEURAL NETWORK MODELS

#Simple RNN model
start_time = time.time() #timing the model
model = Sequential()
model.add(SimpleRNN(75,input_dim=1))  #75 hidden neurons
model.add(Dropout(0.2))   #to reduce overfitting
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

#fit the model
model.fit(X_train_std, y_train, epochs=40, batch_size=1)
end_time = time.time()
print("Elapsed time was %g seconds" % (end_time - start_time))

#predicting the movement of the next 99 days of SPY
y_pred_model = model.predict(X_test_std)[1:100]
y_pred_model[1:]=y_pred_model[1:]-y_pred_model[0:-1] 
y_pred_model[0]=.000001
y_pred_model = np.where(y_pred_model >= 0.0, 1, 0)

#Evaluation Report
print('Misclassified samples: %d' % (y_test[1:100] != y_pred_model[:,0]).sum())
print('Accuracy: %.2f' % accuracy_score(y_test[1:100], y_pred_model))
print(confusion_matrix(y_test[1:100],y_pred_model))
print(classification_report(y_test[1:100],y_pred_model))


#For LSTM 
#Simply change the SimpleRNN in the code above to LSTM.

#GRU models,
#Simply change the SimpleRNN in the code above to GRU.


#------------------------------------------
#After performing the 20 experiments for each 
#of SimpleRNN, LSTM and GRU

#plot for accuracy of experiments
experiments = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
x = np.array(experiments)

y = [0.73,0.73,0.70,0.70,0.70,0.74,0.72,0.72,0.71,0.72,
     0.75,0.73,0.74,0.74,0.72,0.74,0.72,0.74,0.73,0.75]
z = [0.73,0.74,0.73,0.73,0.71,0.74,0.73,0.71,0.69,0.70,
    0.71,0.71,0.69,0.70,0.71,0.73,0.71,0.69,0.69,0.66]
k = [0.71,0.73,0.72,0.71,0.71,0.74,0.70,0.72,0.72,0.70,
    0.73,0.71,0.68,0.70,0.70,0.73,0.69,0.69,0.69,0.69]

ax = plt.subplot(111)
ax.bar(x-0.25, y,width=0.25,color='b',align='center',label='Simple RNN')
ax.bar(x, z,width=0.25,color='g',align='center',label='LSTM')
ax.bar(x+0.25, k,width=0.25,color='r',align='center',label ='GRU')
plt.xlabel('Experiment')
plt.ylabel('Accuracy')
plt.title('Accuracy of Experiments')
plt.legend(loc='upper left', bbox_to_anchor=(1, 1))
plt.xticks(experiments)
plt.savefig('myfig1',bbox_inches='tight')
plt.show()


#plot for GPU time of experiments
y1 = [50.566,97.8223,144.874,187.599,233.552,53.3097,102.235,
     150.565,198.614,247.727,58.6347,177.455,166.386,218.544,
     271.841,63.135,120.761,180.189,237.193,373.436]  #srnn 

z1 = [445.469,880.36,1289.69,1722.03,2692.28,455.511,892.438,
      1328.87,1768.11,2812.48,456.057,889.301,1336.44,
      1812.23,2798.16,483.806,940.876,1409.29,1912.45,2886.09] #lstm

k1 = [449.948,779.411,1146.83,1555.16,1792.24,409.454,810.361,
      1211.28,1588.98,1812.24,411.027,808.906,1195.79,1584.82,
      1884.32,406.104,806.767,1211.98,1601.12,2006.17] #gru

ax = plt.subplot(111)
ax.bar(x-0.25, y1,width=0.25,color='b',align='center',label='Simple RNN')
ax.bar(x, z1,width=0.25,color='g',align='center',label='LSTM')
ax.bar(x+0.25, k1,width=0.25,color='r',align='center',label='GRU')
plt.xlabel('Experiment')
plt.ylabel('Time(seconds)')
plt.title('GPU time of Experiments')
plt.legend(loc='upper left', bbox_to_anchor=(1, 1))
plt.xticks(experiments)
plt.savefig('myfig',bbox_inches='tight')
plt.show()


#-------------------------------------
#Simulating profit
'''
Mov = predicted movement of stock
Inc = Initial capital
Stok = Stock, which is a form of capital
Cash = another form of capital
'''
def simul_profit(Inc,Cash,Stok,Mov):
    Cash, Stok = Inc, 0
    for i in range(len(Mov)):
        if Mov[i]==1:
            if Stok==0:
                s=Cash/Cp[i]
                Stok=int(s)
                Cash=(s-Stok)*Cp[i]
        else:
            if Stok>0:
                Cash +=Stok*Cp[i]
                Stok=0
    Cash+=Stok*Cp[-1]
    print(Cash-Inc)